In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os

# Ruta al archivo JSON con las credenciales de la cuenta de servicio
KEY_FILE_LOCATION = os.path.join(os.getcwd(), 'credenciales.json')

# ID de la vista de Google Analytics
VIEW_ID = '92320289'

# Configurar las credenciales
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE_LOCATION,
    scopes=['https://www.googleapis.com/auth/analytics.readonly']
)

# Crear un servicio de Google Analytics
analytics = build('analyticsreporting', 'v4', credentials=credentials)

try:
    # Realizar una consulta de ejemplo
    response = analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                    'viewId': VIEW_ID,
                    'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
                    'metrics': [{'expression': 'ga:sessions'}],
                    'dimensions': [{'name': 'ga:browser'}]
                }
            ]
        }
    ).execute()
    # Imprimir la respuesta
    print(response)
except Exception as e:
    # Imprimir información de error
    print(f"Error: {e}")

Error: <HttpError 403 when requesting https://analyticsreporting.googleapis.com/v4/reports:batchGet?alt=json returned "Analytics Reporting API has not been used in project 250391785256 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/analyticsreporting.googleapis.com/overview?project=250391785256 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.". Details: "[{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developers console API activation', 'url': 'https://console.developers.google.com/apis/api/analyticsreporting.googleapis.com/overview?project=250391785256'}]}, {'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'SERVICE_DISABLED', 'domain': 'googleapis.com', 'metadata': {'consumer': 'projects/250391785256', 'service': 'analyticsreporting.googleapis.com'}}]">
